In [16]:
from tg.grammar_ru import *
from tg.grammar_ru.features import *
import pandas as pd
import numpy as np

def _get_pronouns(db: DataBundle):
    morphology_df = db.pymorphy
    pronouns_filter = morphology_df['normal_form'].isin(['он', 'она', 'оно'])
    pronouns_df = morphology_df[pronouns_filter][['gender']]
    pronouns_df = pronouns_df.reset_index()
    annotated_filter = db.src[db.src.word_id.isin(pronouns_df.word_id)].category > 0
    pronouns_df = pronouns_df[annotated_filter.reset_index(drop=True)]
    return pronouns_df.add_prefix('pronoun_')

def _get_antecedent_candidates(db: DataBundle):
    cand_filter = (db.slovnet['POS'].isin(['NOUN', 'PROPN']) &
                   (db.slovnet['Number'] == 'Sing')).reset_index(drop=True)
    candidates_df = db.pymorphy[cand_filter][['gender']]
    return candidates_df[['gender']].reset_index().add_prefix('candidate_')

def _filter_candidates(pronouns_df: pd.DataFrame, candidates_df: pd.DataFrame, max_candidates):
    merged_df = pronouns_df.merge(candidates_df, how='cross')
    merged_df = merged_df[
        (merged_df['pronoun_word_id'] > merged_df['candidate_word_id']) &
        (merged_df['pronoun_gender'] == merged_df['candidate_gender'])]
    merged_df = merged_df.drop(
        columns=['pronoun_gender', 'candidate_gender']).reset_index(
        drop=True)
    merged_df['candidate_distance'] = merged_df.groupby(['pronoun_word_id']).cumcount(ascending=False)
    merged_df = merged_df[merged_df['candidate_distance'] < max_candidates]
    return merged_df[['pronoun_word_id',
                      'candidate_word_id',
                      'candidate_distance']].reset_index(drop=True)

def get_candidates(db: DataBundle, max_candidates: int):
    chosen_pronouns_df = _get_pronouns(db)
    antecedent_candidates_df = _get_antecedent_candidates(db)
    filtered_df = _filter_candidates(chosen_pronouns_df, antecedent_candidates_df, max_candidates)
    parent_ids = db.slovnet[
            db.slovnet.index.isin(filtered_df['pronoun_word_id'])]['syntax_parent_id']
    parent_df = (parent_ids
                 .to_frame()
                 .reset_index()
                 .rename(columns={'syntax_parent_id': 'pronoun_parent_id', 'word_id': 'pronoun_word_id'}))
    return filtered_df.merge(parent_df, on='pronoun_word_id')

In [7]:
rucoco_dir = Loc.data_cache_path/'rucoco'
import os

def get_frames():
    for entry in os.scandir(rucoco_dir/'frames'):
        yield pd.read_csv(entry).drop(columns=['Unnamed: 0'])
for i in get_frames():
    frame = i
    break

word_to_normal_form = DataBundle.load(rucoco_dir/'bundle-full.zip').words.set_index('word').nf_n.to_dict()
db = DataBundle(src=frame)
pmph = PyMorphyFeaturizer()
pmph.featurize(db)
slvnt = SlovnetFeaturizer()
slvnt.featurize(db)
candidates = get_candidates(db, 5)

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\2026901016.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]


In [8]:
candidates.head(7)

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id
0,166,64,4,169
1,166,67,3,169
2,166,98,2,169
3,166,133,1,169
4,166,159,0,169
5,224,202,4,225
6,224,206,3,225


In [9]:
vectors_file = rucoco_dir/'glove'/'output.txt'
vdf = pd.read_csv(vectors_file, sep=' ',header=None).set_index(0)
vdf = vdf.reset_index(drop=True).iloc[:-1]
vdf.columns=list(range(vdf.shape[1]))
vdf.index.name='index'

In [13]:
def get_context_words(db, with_parents_df, neighb_count=1):
    extended_df = with_parents_df.copy()
    
    for i in range(1, neighb_count + 1):
        left_id = 'left_id_' + str(i)
        extended_df[left_id] = with_parents_df['pronoun_word_id'].apply(
            lambda x: x - i if (x - i >= 0 and db.src.loc[x - i].word_type != 'punct') else -1)
        right_id = 'right_id_' + str(i)
        extended_df[right_id] = with_parents_df['pronoun_word_id'].apply(
            lambda x: x + i if (x + i < len(db.src.index) and db.src.loc[x + i].word_type != 'punct') else -1)
        
        extended_df = extended_df.merge(
            db.src.word.str.lower().rename('left_word_' + str(i)),
            left_on=left_id, right_index=True, how='left').reset_index(drop=True)
        extended_df = extended_df.merge(
            db.src.word.str.lower().rename('right_word_' + str(i)),
            left_on=right_id, right_index=True, how='left').reset_index(
            drop=True)
    extended_df = extended_df.merge(
            db.src.word.str.lower().rename('parent_word'),
            left_on='pronoun_parent_id', right_index=True, how='left')
    return extended_df
        
def get_nf_n(word):
    try:
        return word_to_normal_form[word]
    except KeyError:
        return 0
        
def get_glove_prod_e_norm(df, col1, col2):
    copy = df.copy()
    nf1 = copy[col1].apply(lambda a: get_nf_n(a))
    nf2 = copy[col2].apply(lambda a: get_nf_n(a))
    v1 = vdf.iloc[nf1].reset_index(drop=True)
    v2 = vdf.iloc[nf2].reset_index(drop=True)
    sc_prod = (v1 * v2).sum(axis=1)
    mods_prod = ((v1 * v1).sum(axis=1) ** 0.5) * ((v2 * v2).sum(axis=1) ** 0.5)
    prods = sc_prod / mods_prod
    copy['prod_e'] = prods.apply(lambda x: np.e ** x)
    return copy.groupby('pronoun_word_id')['prod_e'].transform(lambda x: x / x.sum()).fillna(0)

def get_result_score(db, with_parents_df, neighb_count=1, no_parent=False, coeff=1):
    with_parents_df = (with_parents_df.merge(db.src[['word_id', 'word']], left_on='candidate_word_id', right_on='word_id', how='left')
                       .rename(columns={'word': 'candidate_word'}))
    extended_df = get_context_words(db, with_parents_df, neighb_count)
    result_series = get_glove_prod_e_norm(extended_df, 'candidate_word', 'parent_word')
    if no_parent:
        result_series['score'] = 0
    if neighb_count == 0:
        return result_series
    for i in range(1, neighb_count + 1):
        i_addendum = (get_glove_prod_e_norm(extended_df,
                                            'candidate_word',
                                            'left_word_' + str(i)) +
                      get_glove_prod_e_norm(extended_df,
                                            'candidate_word',
                                            'right_word_' + str(i)))
        result_series += i_addendum * coeff
    return result_series

In [207]:
filtered_df = candidates.copy()
filtered_df['score'] = get_result_score(db, candidates.copy(), 0)
result = filtered_df.loc[filtered_df.groupby('pronoun_word_id')['score'].idxmax()].reset_index(drop=True)
result['pronoun_category'] = db.src.loc[db.src.word_id.isin(result.pronoun_word_id)].category.reset_index(drop=True)
result['candidate_category'] = db.src.loc[db.src.word_id.isin(result.candidate_word_id)].category.reset_index(drop=True)
result

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,score,pronoun_category,candidate_category
0,166,159,0,169,0.252534,7.0,6.0
1,224,206,3,225,0.322309,8.0,0.0
2,265,255,1,266,0.214738,3.0,0.0
3,283,281,0,284,0.377052,3.0,0.0
4,336,300,3,337,0.331338,9.0,9.0
5,358,348,2,360,0.299635,7.0,11.0


In [208]:
(result.pronoun_category == result.candidate_category).sum()

1

In [209]:
filtered_df = candidates.copy()
filtered_df['score'] = get_result_score(db, candidates.copy(), 5)
result = filtered_df.loc[filtered_df.groupby('pronoun_word_id')['score'].idxmax()].reset_index(drop=True)
result['pronoun_category'] = db.src.loc[db.src.word_id.isin(result.pronoun_word_id)].category.reset_index(drop=True)
result['candidate_category'] = db.src.loc[db.src.word_id.isin(result.candidate_word_id)].category.reset_index(drop=True)
result

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,score,pronoun_category,candidate_category
0,166,67,3,169,2.283223,7.0,0.0
1,224,206,3,225,2.932779,8.0,0.0
2,265,242,2,266,2.490820,3.0,0.0
3,283,265,3,284,2.586753,3.0,3.0
4,336,300,3,337,3.088569,9.0,9.0
5,358,348,2,360,2.862518,7.0,11.0


In [210]:
(result.pronoun_category == result.candidate_category).sum()

2

In [281]:
pmph = PyMorphyFeaturizer()
slvnt = SlovnetFeaturizer()
total = 0
success = 0

for frame in get_frames():
    db = DataBundle(src=frame)
    pmph.featurize(db)
    slvnt.featurize(db)
    candidates = get_candidates(db, 5)
    candidates['score'] = get_result_score(db, candidates, 0)
    result = candidates.loc[candidates.groupby('pronoun_word_id')['score'].idxmax()].reset_index(drop=True)
    result = result.merge(db.src[['word_id', 'category']], left_on='pronoun_word_id', right_on='word_id')
    result = result.merge(db.src[['word_id', 'category']], left_on='candidate_word_id', right_on='word_id')
    total += len(result)
    success += (result.category_x == result.category_y).sum()

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

...

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

In [282]:
total, success

(9311, 1646)

In [283]:
total = 0
success = 0

for frame in get_frames():
    db = DataBundle(src=frame)
    pmph.featurize(db)
    slvnt.featurize(db)
    candidates = get_candidates(db, 5)
    candidates['score'] = get_result_score(db, candidates, 5)
    result = candidates.loc[candidates.groupby('pronoun_word_id')['score'].idxmax()].reset_index(drop=True)
    result = result.merge(db.src[['word_id', 'category']], left_on='pronoun_word_id', right_on='word_id', how='left')
    result = result.merge(db.src[['word_id', 'category']], left_on='candidate_word_id', right_on='word_id', how='left')
    total += len(result)
    success += (result.category_x == result.category_y).sum()

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

...

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

In [284]:
total, success

(9328, 2022)

In [251]:
pronouns_total = 0
correct_candidates = 0

for frame in get_frames():
    db = DataBundle(src=frame)
    pmph.featurize(db)
    slvnt.featurize(db)
    candidates = get_candidates(db, 5)
    candidates = candidates.merge(db.src[['word_id', 'category']], left_on='pronoun_word_id', right_on='word_id')
    candidates = candidates.merge(db.src[['word_id', 'category']], left_on='candidate_word_id', right_on='word_id')
    candidates['match'] = candidates.category_x == candidates.category_y
    correct_candidates += candidates.groupby('pronoun_word_id')['match'].any().sum()
    pronouns_total += len(candidates.groupby('pronoun_word_id'))

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

...

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

In [252]:
pronouns_total, correct_candidates

(8907, 5093)

Remove bad candidate pronouns

In [277]:
total = 0
success = 0

for frame in get_frames():
    db = DataBundle(src=frame)
    pmph.featurize(db)
    slvnt.featurize(db)
    candidates = get_candidates(db, 5)
    candidates = candidates.merge(db.src[['word_id', 'category']], left_on='pronoun_word_id', right_on='word_id')
    candidates = candidates.merge(db.src[['word_id', 'category']], left_on='candidate_word_id', right_on='word_id')
    candidates['match'] = candidates.category_x == candidates.category_y
    matches = candidates.groupby('pronoun_word_id')['match'].any()
    candidates = candidates[candidates.pronoun_word_id.isin(matches[matches].index)].reset_index(drop=True)
    
    candidates['score'] = get_result_score(db, candidates, 5)
    result = candidates.loc[candidates.groupby('pronoun_word_id')['score'].idxmax()].reset_index(drop=True)
    total += len(result)
    success += result.match.sum()

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

...

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7236\3134975875.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['

In [278]:
total, success

(5093, 2039)

In [18]:
def fixed_get_glove_prod_e_norm(df, col1, col2):
    copy = df.copy()
    nf1 = copy[col1].apply(lambda a: get_nf_n(a))
    nf2 = copy[col2].apply(lambda a: get_nf_n(a))
    v1 = vdf.iloc[nf1][:300].reset_index(drop=True)
    v2 = vdf.iloc[nf2][301:601].reset_index(drop=True)
    sc_prod = (v1 * v2).sum(axis=1)
    mods_prod = ((v1 * v1).sum(axis=1) ** 0.5) * ((v2 * v2).sum(axis=1) ** 0.5)
    prods = sc_prod / mods_prod
    copy['prod_e'] = prods.apply(lambda x: np.e ** x)
    return copy.groupby('pronoun_word_id')['prod_e'].transform(lambda x: x / x.sum()).fillna(0)

def fixed_get_result_score(db, with_parents_df, neighb_count=1, no_parent=False, coeff=1):
    with_parents_df = (with_parents_df.merge(db.src[['word_id', 'word']], left_on='candidate_word_id', right_on='word_id', how='left')
                       .rename(columns={'word': 'candidate_word'}))
    extended_df = get_context_words(db, with_parents_df, neighb_count)
    result_series = fixed_get_glove_prod_e_norm(extended_df, 'candidate_word', 'parent_word')
    if no_parent:
        result_series['score'] = 0
    if neighb_count == 0:
        return result_series
    for i in range(1, neighb_count + 1):
        i_addendum = (fixed_get_glove_prod_e_norm(extended_df,
                                                  'candidate_word',
                                                  'left_word_' + str(i)) +
                      fixed_get_glove_prod_e_norm(extended_df,
                                                  'candidate_word',
                                                  'right_word_' + str(i)))
        result_series += i_addendum * coeff
    return result_series

In [19]:
total = 0
success = 0

for frame in get_frames():
    db = DataBundle(src=frame)
    pmph.featurize(db)
    slvnt.featurize(db)
    candidates = get_candidates(db, 5)
    candidates = candidates.merge(db.src[['word_id', 'category']], left_on='pronoun_word_id', right_on='word_id')
    candidates = candidates.merge(db.src[['word_id', 'category']], left_on='candidate_word_id', right_on='word_id')
    candidates['match'] = candidates.category_x == candidates.category_y
    matches = candidates.groupby('pronoun_word_id')['match'].any()
    candidates = candidates[candidates.pronoun_word_id.isin(matches[matches].index)].reset_index(drop=True)
    
    candidates['score'] = fixed_get_result_score(db, candidates, 5)
    result = candidates.loc[candidates.groupby('pronoun_word_id')['score'].idxmax()].reset_index(drop=True)
    total += len(result)
    success += result.match.sum()

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gende

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gende

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gende

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gende

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gende

...

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gende

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gende

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gende

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gende

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gender']]
C:\Users\Alexandra\AppData\Local\Temp\ipykernel_7680\260469544.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidates_df = db.pymorphy[cand_filter][['gende

In [20]:
total, success

(5093, 1152)